# Churn Case Part 2

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn=sqlite3.connect("churn.db")

In [3]:
df=pd.read_sql("Select * from churn_all",conn)

In [15]:
df.head()
df.dtypes

CustomerID           object
Gender               object
SeniorCitizen        object
Partner              object
Dependents           object
State                object
Latitude            float64
Longitude           float64
ZipCode               int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Tenure                int64
Contract             object
PaymentMethod        object
PaperlessBilling     object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [14]:
df["TotalCharges"]=pd.to_numeric(df.TotalCharges,errors='coerce')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   State             7043 non-null   object 
 6   Latitude          7043 non-null   float64
 7   Longitude         7043 non-null   float64
 8   ZipCode           7043 non-null   int64  
 9   PhoneService      7043 non-null   object 
 10  MultipleLines     7043 non-null   object 
 11  InternetService   7043 non-null   object 
 12  OnlineSecurity    7043 non-null   object 
 13  OnlineBackup      7043 non-null   object 
 14  DeviceProtection  7043 non-null   object 
 15  TechSupport       7043 non-null   object 
 16  StreamingTV       7043 non-null   object 


In [17]:
df.columns

Index(['CustomerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'State', 'Latitude', 'Longitude', 'ZipCode', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Tenure', 'Contract', 'PaymentMethod', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

## 1. For those customers who unsubscribed the service, what was their average LTV? and how long did they usually stay in the service?

In [18]:
## Extract those who unsubscribed the serviced (Churn = Yes)
churn_df=pd.read_sql("Select * from churn_all where Churn='Yes'",conn)
churn_df

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.30,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,Yes,...,Yes,Yes,No,12,One year,Electronic check,Yes,59.80,727.80,Yes
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,Yes,...,No,No,No,9,Month-to-month,Bank transfer (automatic),Yes,44.20,403.35,Yes
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,Yes,...,No,No,No,1,Month-to-month,Electronic check,Yes,75.75,75.75,Yes
1867,0639-TSIQW,Female,No,No,No,California,33.972293,-116.654195,92282,Yes,...,No,Yes,No,67,Month-to-month,Credit card (automatic),Yes,102.95,6886.25,Yes


In [19]:
# Examine the distribution of TotalCharges
churn_df["TotalCharges"].describe()

count    1869.000000
mean     1531.796094
std      1890.822994
min        18.850000
25%       134.500000
50%       703.550000
75%      2331.300000
max      8684.800000
Name: TotalCharges, dtype: float64

In [23]:
# Find the 80th percentile of the data in TotalCharges
churn_df.TotalCharges.quantile(0.8)

2840.4100000000003

In [25]:
churn_df["TotalCharges"].quantile(0.8)

2840.4100000000003

In [27]:
pd.read_sql("Select * from churn_all where TotalCharges <= 2840.4100000000003",conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
3,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,Yes,...,Yes,No,No,10,Month-to-month,Credit card (automatic),No,55.20,528.35,Yes
4,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,No,...,No,No,Yes,1,Month-to-month,Electronic check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4734,8456-QDAVC,Male,No,No,No,California,32.852947,-114.850784,92283,Yes,...,No,Yes,No,19,Month-to-month,Bank transfer (automatic),Yes,78.70,1495.10,No
4735,7750-EYXWZ,Female,No,No,No,California,34.159534,-116.425984,92284,No,...,Yes,Yes,Yes,12,One year,Electronic check,No,60.65,743.30,No
4736,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,Yes,...,No internet service,No internet service,No internet service,72,Two year,Bank transfer (automatic),Yes,21.15,1419.40,No
4737,6840-RESVB,Male,No,Yes,Yes,California,34.667815,-117.536183,92301,Yes,...,Yes,Yes,Yes,24,One year,Mailed check,Yes,84.80,1990.50,No


In [33]:
churn_df.query("TotalCharges<=2840.4100000000003")

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
5,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,Yes,...,Yes,No,No,10,Month-to-month,Credit card (automatic),No,55.20,528.35,Yes
6,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,No,...,No,No,Yes,1,Month-to-month,Electronic check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,1122-JWTJW,Male,No,Yes,Yes,California,32.698964,-115.886656,92259,Yes,...,No,No,No,1,Month-to-month,Mailed check,Yes,70.65,70.65,Yes
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,Yes,...,Yes,Yes,No,12,One year,Electronic check,Yes,59.80,727.80,Yes
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,Yes,...,No,No,No,9,Month-to-month,Bank transfer (automatic),Yes,44.20,403.35,Yes
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,Yes,...,No,No,No,1,Month-to-month,Electronic check,Yes,75.75,75.75,Yes


In [34]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
total_charges_above80=churn_df.query("TotalCharges>2840.4100000000003")
total_charges_under80=churn_df.query("TotalCharges<=2840.4100000000003")

In [36]:
# Show the distribution of people under the 80th percentile
total_charges_under80.TotalCharges.describe()

count    1495.000000
mean      711.265819
std       766.848197
min        18.850000
25%        85.025000
50%       371.650000
75%      1128.225000
max      2839.650000
Name: TotalCharges, dtype: float64

In [37]:
# Show the distribution of people above the 80th percentile
total_charges_above80.TotalCharges.describe()

count     374.000000
mean     4811.723262
std      1436.724288
min      2841.550000
25%      3522.462500
50%      4571.100000
75%      5891.212500
max      8684.800000
Name: TotalCharges, dtype: float64

# Analysis of Lifetime value Part 2

In [38]:
# Show the distribution of the tenure of people under the 80th percentile
total_charges_under80.Tenure.describe()

count    1495.000000
mean        9.935117
std        10.742349
min         1.000000
25%         1.000000
50%         6.000000
75%        15.000000
max        61.000000
Name: Tenure, dtype: float64

In [39]:
# Show the distribution of the tenure of people above the 80th percentile
total_charges_above80.Tenure.describe()

count    374.000000
mean      50.133690
std       12.334841
min       27.000000
25%       40.000000
50%       49.500000
75%       60.000000
max       72.000000
Name: Tenure, dtype: float64

The average LTV of the 80% of those who unsubscribed is 711, and their tenure is near 10 months. On the hand, the average LTV of the 20% of those who unsubscribed is 4811, and their tenure is near 50 months.

## 2. What kinds of services they subscribed when they were still a customer?

### PhoneService 

In [45]:
#under 80
pd.read_sql("Select PhoneService,Count(*) from churn_all where TotalCharges <= 2840.41 and Churn = 'Yes' group by PhoneService",conn)

,PhoneService,Count(*)
0,No,161
1,Yes,1334


In [50]:
df["Gender"].value_counts()

Gender
Male      3555
Female    3488
Name: count, dtype: int64

In [54]:
total_charges_under80.PhoneService.value_counts()

PhoneService
Yes    1334
No      161
Name: count, dtype: int64

In [55]:
total_charges_under80["PhoneService"].value_counts()

PhoneService
Yes    1334
No      161
Name: count, dtype: int64

In [56]:
# Example of shape
df.shape

(7043, 25)

In [61]:
# Under 80 (percentage)
total_charges_under80.PhoneService.value_counts() / total_charges_under80.shape[0]

PhoneService
Yes    0.892308
No     0.107692
Name: count, dtype: float64

In [62]:
# Above 80 (percentage)
total_charges_above80.PhoneService.value_counts() / total_charges_above80.shape[0]

PhoneService
Yes    0.975936
No     0.024064
Name: count, dtype: float64

For the top 20% (high LTV) only 2% of them didn't use the phone service. On the other hand, lower 80% (LTV) had around 11% of the people that didn't use the phone service. That is around 5 times more as the top 20%. Apparently if you are not subscribed to the phone service you are more likely to be in the bottom 80% of the LTV. We as a company can try to sell this service to more people.

### MutipleLines

In [63]:
total_charges_under80_use_phone = total_charges_under80.query("PhoneService == 'Yes'")

total_charges_under80_use_phone.MultipleLines.value_counts() / total_charges_under80_use_phone.shape[0]

MultipleLines
No     0.590705
Yes    0.409295
Name: count, dtype: float64

In [64]:
total_charges_above80_use_phone = total_charges_above80.query("PhoneService == 'Yes'")

total_charges_above80_use_phone.MultipleLines.value_counts() / total_charges_above80_use_phone.shape[0]

MultipleLines
Yes    0.832877
No     0.167123
Name: count, dtype: float64

For the top 20% of LTV who used the phone service 84% of them used multiple lines, which is 2 times as much as the proportion of people in the low 80% LTV that is around 40%. Having multiple lines is going to get you closer to be in the top 20% of the LTV.

### InternetService

In [66]:
# Under 80 (percentage)
total_charges_under80.InternetService.value_counts() / total_charges_under80.shape[0]

InternetService
Fiber optic    0.640803
DSL            0.283612
No             0.075585
Name: count, dtype: float64

In [67]:
# Above 80 (percentage)
total_charges_above80.InternetService.value_counts() / total_charges_above80.shape[0]

InternetService
Fiber optic    0.906417
DSL            0.093583
Name: count, dtype: float64

All the people in the high 20% LTV used internet service. On the other hand 8% of the people in the low 80th percentile of the LTV did not have internet connection. Also, in the top 20% (high LTV) 90% of them had fiber optic, in comparison to only 64% in the low LTV. So as a company if we want to increase the LTV of customers, we should recommend using internet and Fiber Optic connection.

### Other services

In [69]:
import numpy as np

In [70]:
total_charges_under80_use_internet = total_charges_under80.query('InternetService!="No"')
proportion_internet_sub_service_under80 = np.array([total_charges_under80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0]])

In [71]:
proportion_internet_sub_service_under80

array([0.1316932 , 0.13603473, 0.22720695, 0.23516643, 0.37698987,
       0.3813314 ])

In [76]:
total_charges_above80_use_internet = total_charges_above80.query('InternetService!="No"')
proportion_internet_sub_service_above80 = np.array([total_charges_above80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0]])

In [73]:
proportion_internet_sub_service_above80

array([0.30213904, 0.32620321, 0.55882353, 0.58823529, 0.78342246,
       0.77807487])

In the top 20% LTV, both streaming movies and streaming tv percentages are around 80%, in comparison in the low 80% LTV they are around 40%. Device protection and online backup for the top 20% LTV are around 55%, in comparison to only around 22% in the low LTV. As a company we would like to sell to customers streaming TV and movies and also online backup and device protection if we want to increase LTV.

## 3. For those who churned what is the proportion of each kind of contract?

In [77]:
# Under 80 (percentage)
total_charges_under80.Contract.value_counts() / total_charges_under80.shape[0]

Contract
Month-to-month    0.959197
One year          0.036789
Two year          0.004013
Name: count, dtype: float64

In [78]:
# Above 80 (percentage)
total_charges_above80.Contract.value_counts() / total_charges_above80.shape[0]

Contract
Month-to-month    0.590909
One year          0.296791
Two year          0.112299
Name: count, dtype: float64

## 4. For those who did not churn what is the proportion of each kind of contract?

In [79]:
## Extract those who unsubscribed the services (Churn = Yes)

paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)

In [80]:
paying_df.TotalCharges.describe()

count     5174
unique    4966
top           
freq        11
Name: TotalCharges, dtype: object

In [82]:
paying_df["TotalCharges"]=pd.to_numeric(paying_df.TotalCharges,errors='coerce')

In [83]:
paying_df.TotalCharges.describe()

count    5163.000000
mean     2555.344141
std      2329.456984
min        18.800000
25%       577.825000
50%      1683.600000
75%      4264.125000
max      8672.450000
Name: TotalCharges, dtype: float64

In [84]:
# Find the 80th percentile of the data in TotalCharges
paying_df.TotalCharges.quantile(0.8)

4895.85

In [85]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
paying_total_charges_under80 = paying_df.query("TotalCharges <= 4895.8")
paying_total_charges_above80 = paying_df.query("TotalCharges > 4895.8")

In [86]:
# Under 80 (percentage)
paying_total_charges_under80.Contract.value_counts() / paying_total_charges_under80.shape[0]

Contract
Month-to-month    0.500726
Two year          0.253269
One year          0.246005
Name: count, dtype: float64

In [87]:
# Above 80 (percentage)
paying_total_charges_above80.Contract.value_counts() / paying_total_charges_above80.shape[0]

Contract
Two year          0.572120
One year          0.280736
Month-to-month    0.147144
Name: count, dtype: float64

- If we want to stop Churning we would like to sell contracts to people for at least one year, and much better for 2 years (Conjecture).
- We have 57% of the people that stayed in the company in the top 20% LTV with 2 year contracts, and only 11% of the people in the top 20% had a 2 year contract and left the company.
- 95% of the people that left the company and are in the low 80% have a month-to-month contract, in contrast to only 50% of the people in the low 80% that stayed in the company. We can say that having a month-to-month contract is one of the reasons that people are leaving the company (conjecture).